In [2]:

import logging
import os
import sys
import tempfile
from glob import glob

import torch
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import ArrayDataset, create_test_image_2d, decollate_batch
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandRotate90,
    RandSpatialCrop,
    ScaleIntensity,
    EnsureType,
)
from monai.visualize import plot_2d_or_3d_image


In [4]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 0.10.dev2228
Numpy version: 1.23.1
Pytorch version: 1.12.0+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: b287dfc6a7c397523ba67e10a0436099fb0a44f7
MONAI __file__: C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.2.0
Tensorboard version: 2.9.1
gdown version: 4.5.1
TorchVision version: 0.13.0+cu113
tqdm version: 4.64.0
lmdb version: 1.3.0
psutil version: 5.9.1
pandas version: 1.4.3
einops version: 0.4.1
transformers version: 4.20.1
mlflow version: 1.27.0
pynrrd version: 0.4.3

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [6]:
imgs = sorted(glob(os.path.join("imagesTr/*.nii.gz")))
segmentations = sorted(glob(os.path.join("labelsTr/*.nii.gz")))
print(imgs)
print(segmentations)

['imagesTr\\amos_0001.nii.gz', 'imagesTr\\amos_0004.nii.gz', 'imagesTr\\amos_0005.nii.gz', 'imagesTr\\amos_0006.nii.gz', 'imagesTr\\amos_0007.nii.gz', 'imagesTr\\amos_0009.nii.gz', 'imagesTr\\amos_0010.nii.gz', 'imagesTr\\amos_0011.nii.gz', 'imagesTr\\amos_0014.nii.gz', 'imagesTr\\amos_0015.nii.gz', 'imagesTr\\amos_0016.nii.gz', 'imagesTr\\amos_0017.nii.gz', 'imagesTr\\amos_0019.nii.gz', 'imagesTr\\amos_0021.nii.gz', 'imagesTr\\amos_0023.nii.gz', 'imagesTr\\amos_0024.nii.gz', 'imagesTr\\amos_0025.nii.gz', 'imagesTr\\amos_0027.nii.gz', 'imagesTr\\amos_0030.nii.gz', 'imagesTr\\amos_0033.nii.gz', 'imagesTr\\amos_0035.nii.gz', 'imagesTr\\amos_0036.nii.gz', 'imagesTr\\amos_0038.nii.gz', 'imagesTr\\amos_0042.nii.gz', 'imagesTr\\amos_0043.nii.gz', 'imagesTr\\amos_0044.nii.gz', 'imagesTr\\amos_0045.nii.gz', 'imagesTr\\amos_0047.nii.gz', 'imagesTr\\amos_0048.nii.gz', 'imagesTr\\amos_0049.nii.gz', 'imagesTr\\amos_0050.nii.gz', 'imagesTr\\amos_0052.nii.gz', 'imagesTr\\amos_0054.nii.gz', 'imagesTr

In [8]:
# define transforms for image and segmentation
train_imtrans = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        ScaleIntensity(),
        RandSpatialCrop((96, 96), random_size=False),
        RandRotate90(prob=0.5, spatial_axes=(0, 1)),
        EnsureType(),
    ]
)
train_segtrans = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        ScaleIntensity(),
        RandSpatialCrop((96, 96), random_size=False),
        RandRotate90(prob=0.5, spatial_axes=(0, 1)),
        EnsureType(),
    ]
)

In [9]:
val_imtrans = Compose([LoadImage(image_only=True), AddChannel(), ScaleIntensity(), EnsureType()])
val_segtrans = Compose([LoadImage(image_only=True), AddChannel(), ScaleIntensity(), EnsureType()])

In [12]:
# define array dataset, data loader
check_ds = ArrayDataset(imgs, train_imtrans, segmentations, train_segtrans)
check_loader = DataLoader(check_ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = monai.utils.misc.first(check_loader)
print(im.shape, seg.shape)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\transform.py", line 89, in apply_transform
    return _apply_transform(transform, data, unpack_items)
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\transform.py", line 53, in _apply_transform
    return transform(parameters)
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\croppad\array.py", line 605, in __call__
    self.randomize(img.shape[1:])
  File "C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\transforms\croppad\array.py", l